In [12]:
from importlib.metadata import version
from tqdm import tqdm

In [15]:
import urllib.request
import json

def query_model(prompt, model="llama3.1", url="http://localhost:11434/api/chat"):
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "options": {
            "seed": 123,
            "temperature": 0,
        }
    }
    payload = json.dumps(data).encode("utf-8")
    request = urllib.request.Request(url, data=payload, method='POST')
    request.add_header("Content-Type", "application/json")

    response_data = ""
    with urllib.request.urlopen(request) as response:
        while True:
            line = response.readline().decode('utf-8')
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json['message']['content']
    
    return response_data

In [18]:
res = query_model('What do Llamas eat?')
print(res)

Llamas are herbivores, which means they primarily eat plants and plant-based foods. Their diet consists of:

1. **Grasses**: Various types of grasses, including tall grasses, short grasses, and grassy weeds.
2. **Leaves**: Leaves from trees and shrubs, such as willow, alder, and oak.
3. **Hay**: High-quality hay, like timothy or alfalfa hay, is a staple in many llama diets.
4. **Fruits**: Fresh fruits, like apples, berries, and melons, are also consumed by llamas.
5. **Grains**: Some llamas may be fed grains, such as oats or corn, but this should not make up more than 10-20% of their diet.
6. **Browse**: Browse refers to the leaves, twigs, and branches of shrubs and trees. Llamas love to browse on plants like willow, alder, and cedar.

In general, a llama's diet should consist of:

* 50-60% hay
* 20-30% grasses and other forages (like browse)
* 10-20% grains (if fed at all)
* 5-10% fruits and treats

It's essential to provide llamas with high-quality food, fresh water, and a balanced d

In [19]:
from pathlib import Path

json_file = "instruction-data.json"
with open(json_file, "r") as f:
    json_data = json.load(f)

len(json_data)

1100

In [21]:
entry = json_data[0]

In [28]:
instruction_text = (
        f"Below is an instruction that describes a task. Write a response that "
        f"appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

# print(instruction_text)

input_text = f"\n\n### Input:\n{entry['input']}" if entry['input'] else ""
print(instruction_text + input_text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Evaluate the following phrase by transforming it into the spelling given.

### Input:
freind --> friend


In [29]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. Write a response that "
        f"appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )
    input_text = f"\n\n### Input:\n{entry['input']}" if entry['input'] else ""
    return instruction_text + input_text

In [30]:
print(format_input(entry))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Evaluate the following phrase by transforming it into the spelling given.

### Input:
freind --> friend


In [36]:
import random
random.choice(['p', 'np'])

'np'

In [39]:
politeness = random.choice(['p', 'np'])

In [42]:
prompt = (
        f"Given the input `{format_input(entry)}` "
        f"and correct output `{entry['output']}`, "
        f"slightly rewrite the output to be more {politeness}."
        "Keep the modification minimal."
        "Only return the generated response and nothing else."
    )

print(prompt)

Given the input `Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Evaluate the following phrase by transforming it into the spelling given.

### Input:
freind --> friend` and correct output `The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".`, slightly rewrite the output to be more p.Keep the modification minimal.Only return the generated response and nothing else.


In [49]:
for entry in json_data[-5:]:
    politeness = random.choice(["polite", "impolite"])
    prompt = (
        f"Given the input `{format_input(entry)}` "
        f"and correct output `{entry['output']}`, "
        f"completely rewrite the output to be more {politeness}. "
        "Keep the modification minimal."
        "Only return the generated response and nothing else."
    )
    print("\nDataset response:")
    print(">>", entry['output'])
    print(f"{politeness} response:")
    print(">>", query_model(prompt))


Dataset response:
>> False
impolite response:
>> Are you kidding me? The moon is most definitely NOT a planet, that's just basic astronomy 101. False.

Dataset response:
>> 1000 milliliters is equivalent to 1 liter.
polite response:
>> I'd be happy to help you with converting 1000 milliliters to liters! The result of this conversion is that 1000 milliliters is equivalent to 1 liter.

Dataset response:
>> She was singing in the choir.
impolite response:
>> She was bloody well singing in the choir.

Dataset response:
>> A synonym for 'clever' is 'smart.'
polite response:
>> I appreciate your request! I'd like to provide a helpful response. A suitable alternative to "clever" would be "intelligent."

Dataset response:
>> Should you have called me?
polite response:
>> Shouldn't you have called me instead?


In [50]:
def generate_model_responses(json_data):
    for i, entry in enumerate(tqdm(json_data, desc='Creating preference data')):
        politeness = random.choice(["polite", "impolite"])
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"completely rewrite the output to be more {politeness}. "
            "Keep the modification minimal."
            "Only return the generated response and nothing else."
        )
        model_response = query_model(prompt)

        if politeness == "polite":
            json_data[i]['chosen'] = model_response
            json_data[i]['rejected'] = entry['output']
        else:
            json_data[i]['chosen'] = entry['output']
            json_data[i]['rejected'] = model_response

In [51]:
generate_model_responses(json_data)

python(24396) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Creating preference data: 100%|██████████| 1100/1100 [38:30<00:00,  2.10s/it]   


In [52]:
with open('instruction-data-with-preference.json', 'w') as f:
    json.dump(json_data, f, indent=4)